In [ ]:
# matplotlib 한글 깨짐방지
# 아래코드 실행후 '런타임 다시시작' 후 모듈 불러오기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,190 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126102 files and dire

## Import Library

In [ ]:
# 필요한 모듈 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Import Data

In [ ]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/ESAA OB 2조/프로젝트 2/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA OB 2조/프로젝트 2/data/test_x.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA OB 2조/프로젝트 2/data/sample_submission.csv')

In [ ]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


## Preprocessing

지피티의 추천

문체 분석 : 문장 길이, 구두점 사용 빈도, 복잡도

어휘 분석 : tf-idf, 표제어 사용 빈도, 고유 단어 비율

문법 스타일 : 품사 분포, bigram, 구문 구조 패턴

감정적 표현 : 감성 점수

-> 여러 개 만들고 feature importance 출력해서 선택..?

In [ ]:
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
from itertools import islice
import re

nlp = spacy.load("en_core_web_sm")

def pos_bigrams(pos_list):
    return zip(pos_list, islice(pos_list, 1, None))


# 기본적인 전처리
def clean_text(text):
    # 소문자 + 줄바꿈 제거
    text = text.lower().replace('\n', ' ')

    # 숫자 제거
    text = re.sub(r'\d+', '', text)

    # 공백 정리
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 특징 추출(여기 함수에 어떤 내용을 적을지 정해야 함)
def extract_features(text):
    clean = clean_text(text)
    doc = nlp(clean)

    # 불용어 제거 + 표제어 추출 + 알파벳 필터링
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and token.is_alpha and not token.is_punct
    ]
    clean_lemmas = " ".join(tokens)

    # 문장/단어 길이 지표
    sentence_count = len(list(doc.sents))
    word_count = len([token for token in doc if token.is_alpha])
    avg_sentence_length = word_count / sentence_count if sentence_count > 0 else 0
    char_count = len(clean)

    # 품사별 분포
    ## POS 태그 비율
    pos_tags = [token.pos_ for token in doc if token.is_alpha]
    pos_counts = Counter(pos_tags)
    total = sum(pos_counts.values())
    pos_ratio = {f'pos_{k}': v / total for k, v in pos_counts.items()}

    ## POS bigram 비율
    bigrams_list = list(pos_bigrams(pos_tags))
    bigram_counts = Counter(bigrams_list)
    total_bigrams = sum(bigram_counts.values())
    pos_bigram = {
        f'bigram_{a}-{b}': v / total_bigrams for (a, b), v in bigram_counts.items()
    }

    ## 문장 구조 복잡성 지표
    complex_tags = ['ADP', 'DET', 'CCONJ', 'PUNCT']
    complex_count = sum(pos_counts.get(tag, 0) for tag in complex_tags)
    complexity_ratio = {'complex_ratio': complex_count / total if total > 0 else 0}

    # 피처 통합(칼럼 생성)
    features = {
        "clean_text": clean_lemmas,
        "char_count": char_count,
        "word_count": word_count,
        "sentence_count": sentence_count,
        "avg_sentence_length": avg_sentence_length
    }
    features.update(pos_ratio)
    features.update(pos_bigram)
    features.update(complexity_ratio)
    return features

## 전처리 및 피처 엔지니어링

In [ ]:
# 피처 추출
train_feats = train['text'].apply(extract_features)
test_feats = test['text'].apply(extract_features)

train_df = pd.DataFrame(train_feats.tolist())
test_df = pd.DataFrame(test_feats.tolist())

# TF-IDF 벡터화 (clean_text 사용) (희소 행렬로 칼럼 생성)
tfidf = TfidfVectorizer(max_features=3000)
X_train_tfidf = tfidf.fit_transform(train_df['clean_text']).toarray()
X_test_tfidf = tfidf.transform(test_df['clean_text']).toarray()

# 수치 피처 추출 (clean_text 제외)
X_train_num = train_df.drop(columns=['clean_text']).values
X_test_num = test_df.drop(columns=['clean_text']).values

# 스케일링
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

# 벡터 결합
X_train_final = np.hstack([X_train_tfidf, X_train_num_scaled])
X_test_final = np.hstack([X_test_tfidf, X_test_num_scaled])
y_train = train['label']

# 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train_final, y_train)

from sklearn.metrics import log_loss

y_pred_proba = model.predict_proba(X_train_final)

loss = log_loss(y_train, y_pred_proba)
print("Log Loss:", loss)

KeyboardInterrupt: 

## Submission

In [ ]:
test_pred_proba = model.predict_proba(X_test_final)

submission = pd.read_csv("sample_submission.csv")

submission.iloc[:, 1:] = test_pred_proba

submission.to_csv("submission_prob.csv", index=False)